In [ ]:
using Revise

In [ ]:
using DrakeVisualizer
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window()
using RigidBodyDynamics
using StaticArrays
using RigidBodyTreeInspector
using LCPSim
using JuMP, Gurobi

In [ ]:
urdf_mech = parse_urdf(Float64, "box_valkyrie.urdf")
mechanism, base = planar_revolute_base()
attach!(mechanism, base, urdf_mech)
world = root_body(mechanism)

basevis = Visualizer()[:boxval]
delete!(basevis)
vis = basevis[:robot]
setgeometry!(vis, mechanism, parse_urdf("box_valkyrie.urdf", mechanism))

floor = planar_obstacle(default_frame(world), [0, 0, 1.], [0, 0, 0.], 1.)

contact_limbs = findbody.(mechanism, ["rh", "lh", "rf", "lf"])
hands = findbody.(mechanism, ["rh", "lh"])
feet = findbody.(mechanism, ["rf", "lf"])

env = Environment(
    Dict(vcat(
            [body => ContactEnvironment(
                [Point3D(default_frame(body), SVector(0., 0, 0))],
                [floor])
                for body in feet],
            )));

In [ ]:
function dynamics(mechanism, x, u)
    state = MechanismState{eltype(x)}(mechanism)
    result = DynamicsResult{eltype(x)}(mechanism)
    ẋ = zeros(eltype(x), 4)
    dynamics!(ẋ, result, state, x, u)
    ẋ
end

In [ ]:
using PyCall

In [ ]:
py"""
import numpy as np
import scipy.linalg

from copy import copy

def nullspace_basis(A):
    V = np.linalg.svd(A)[2].T
    rank = np.linalg.matrix_rank(A)
    Z = V[:,rank:]
    return clean_matrix(Z)

def rangespace_basis(A):
    Z = nullspace_basis(A.T)
    return nullspace_basis(Z.T)

def linearly_independent_rows(A, tol=1.e-6):
    R = linalg.qr(A.T)[1]
    R_diag = np.abs(np.diag(R))
    return list(np.where(R_diag > tol)[0])

def clean_matrix(M, tol=1.e-9):
    M_clean = copy(M)
    for i in range(M.shape[0]):
        for j in range(M.shape[1]):
            if np.abs(M[i,j]) < tol:
                M_clean[i,j] = 0.
    return M_clean

def dare(A, B, Q, R):
    # cost to go
    P = scipy.linalg.solve_discrete_are(A, B, Q, R)
    # optimal gain
    K = - scipy.linalg.inv(B.T.dot(P).dot(B)+R).dot(B.T).dot(P).dot(A)
    return P, K

def canonical_reachability_decomposition(A, B):
    n = A.shape[0]
    R = np.hstack([np.linalg.matrix_power(A, i).dot(B) for i in range(n)])
    n_R = np.linalg.matrix_rank(R)
    T_R = rangespace_basis(R)
    T_N = nullspace_basis(R.T)
    T = np.hstack((T_R, T_N))
    T_inv = np.linalg.inv(T)
    A_canonical = T_inv.dot(A).dot(T)
    B_canonical = T_inv.dot(B)
    A_RR = A_canonical[:n_R,:n_R]
    A_RN = A_canonical[:n_R,n_R:]
    A_NR = A_canonical[n_R:,:n_R]
    A_NN = A_canonical[n_R:,n_R:]
    B_R = B_canonical[:n_R,:]
    B_N = B_canonical[n_R:,:]
    return {
    'n_R': n_R,
    'T': T, 'T_R': T_R, 'T_N':T_N,
    'A': A_canonical, 'A_RR': A_RR, 'A_RN': A_RN, 'A_NR': A_NR, 'A_NN': A_NN,
    'B': B_canonical, 'B_R': B_R, 'B_N': B_N
    }

def reachable_lqr(A, B, Q, R):
    crd = canonical_reachability_decomposition(
        A,
        B
        )
    n_R, T, T_R, T_N, A_RR, B_R = [crd['n_R'], crd['T'], crd['T_R'], crd['T_N'], crd['A_RR'], crd['B_R']]
    Q_R = T_R.T.dot(Q).dot(T_R)
    P_R, K_R = dare(A_RR, B_R, Q_R, R)
    print A_RR
    print B_R
    print Q_R
    print R
    print K_R
    T_inv = np.linalg.inv(T)
    T_inv_R = T_inv[:n_R,:]
    P = T_inv_R.T.dot(P_R).dot(T_inv_R)
    K = K_R.dot(T_inv_R)
    return K, P

def zero_order_hold(A, B, c, h):

    # system dimensions
    n_x = np.shape(A)[0]
    n_u = np.shape(B)[1]

    # zero order hold
    mat_c = np.zeros((n_x+n_u+1, n_x+n_u+1))
    mat_c[0:n_x,:] = np.hstack((A, B, c))
    mat_d = scipy.linalg.expm(mat_c*h)

    # discrete time dynamics
    A_d = mat_d[0:n_x, 0:n_x]
    B_d = mat_d[0:n_x, n_x:n_x+n_u]
    c_d = mat_d[0:n_x, n_x+n_u:n_x+n_u+1]

    return A_d, B_d, c_d

def continuous_reachable_lqr(A, B, Q, R, h):
    A_d, B_d, c_d = zero_order_hold(A, B, np.zeros((A.shape[0], 1)), h)
    return reachable_lqr(A_d, B_d, Q, R)
"""

const reachable_lqr = py"reachable_lqr"
const continuous_reachable_lqr = py"continuous_reachable_lqr"

In [ ]:
using Plots
using ForwardDiff

Model is a single-leg hopper, with the foot on the ground. Inputs are the force on the leg, with a positive force acting downward

In [ ]:
using ForwardDiff

In [ ]:
x_fp = [1., -1, 0, 0]
u_fp = [-9.81]

function dynamics(x, u)
    q = x[1:2]
    v = x[3:4]
    # assume the foot is stuck to the ground, then we have:
    v̇ = zeros(promote_type(eltype(x), eltype(u)), 2)
    v̇[1] = -u[1] - 9.81
    v̇[2] = -v̇[1]
    vcat(v, v̇)
end

In [ ]:
A = ForwardDiff.jacobian(x -> dynamics(x, u_fp), x_fp)
B = ForwardDiff.jacobian(u -> dynamics(x_fp, u), u_fp)
Q = diagm([100, 10, 1, 1])
R = 0.1 * eye(1)
@show A B Q R

In [ ]:
dynamics(x_fp, u_fp)

In [ ]:
Jc = [1. 1]
Jcdot = [0. 0]

In [ ]:
N = nullspace([Jc zeros(1, 2); Jcdot Jc])

In [ ]:
Am = N' * A * N
Bm = N' * B
Rm = R
Qm = N' * Q * N

In [ ]:
using ControlSystems: lqr

In [ ]:
Km = lqr(Am, Bm, Qm, Rm)

In [ ]:
K = Km * N'

In [ ]:
x = [1.5, -1, 0, 0]
xs = [x]
Δt = 0.001
for i in 1:1000
    u = -K * (xs[end] .- x_fp)
    ẋ = dynamics(xs[end], u)
    push!(xs, xs[end] .+ Δt .* ẋ)
end

In [ ]:
plot([x[1] for x in xs], ylim=(0, 2))